In [1]:
# Flood area mapping with SAR. From article: https:#medium.com/@geonextgis/the-power-of-sar-data-and-google-earth-engine-in-flood-area-mapping-0a148df4c43b
# Import geemap
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [2]:
# Step 1. Import datasets
roi = ee.FeatureCollection("users/geonextgis/Maldah_Projected")
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
srtm = ee.Image("USGS/SRTMGL1_003")

In [3]:
# Step 2. Display the Region of Interest
roi_style = {
  'color': "red",
  'fillColor': "00000000",
  'width': 1.5
}
Map.addLayer(roi.style(**roi_style), {}, "Region of Interest")
Map.centerObject(roi, 10)

In [4]:
# Step 3. Prep Sentinel-1 SAR GRD Imagery
# On 12 August 2021, severe floods affected the Maldah district of West Bengal.
# At least 30 villages in Malda district were inundated.

# Select images by predefined dates
beforeStart = "2021-08-01"
beforeEnd = "2021-08-10"
afterStart = "2021-08-10"
afterEnd = "2021-08-22"

# Import the Sentinel-1 SAR GRD image collection
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")

# Filter the image collection
s1Filtered = s1.filterBounds(roi) \
                   .filter(ee.Filter.eq("instrumentMode", "IW")) \
                   .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH")) \
                   .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
                   .filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING")) \
                   .filter(ee.Filter.eq("resolution_meters", 10)) \
                   .select(["VV", "VH"])

# Before Image Acquistion Date: 2021-08-09
beforeImage = s1Filtered.filterDate(beforeStart, beforeEnd) \
                            .first() \
                            .clip(roi)

# After Image Acquistion Date: 2021-08-21
afterImage = s1Filtered.filterDate(afterStart, afterEnd) \
                           .first() \
                           .clip(roi)

# Create a function to add ratio band
def addRatioBand(image):
  ratioBand = image.select("VV").divide(image.select("VH")).rename("VV/VH")
  return image.addBands(ratioBand)

beforeImage = addRatioBand(beforeImage)
afterImage = addRatioBand(afterImage)

# Define a visualization
visParams = {
  'min': [-25, -25, 0],
  'max': [0, 0, 2]
}
Map.addLayer(beforeImage, visParams, "Before Floods", False)
Map.addLayer(afterImage, visParams, "After Floods", False)

# Visualize the changes by creating a composite image
# Band Combination: VH-before image, VH-after image, and VH-before image
compositeImage = ee.Image.cat([beforeImage.select("VH").rename("VH_Before"),
                                   afterImage.select("VH").rename("VH_After"),
                                   beforeImage.select("VH").rename("VH_Before_2")])
Map.addLayer(compositeImage, {'min': -25, 'max': -8}, "Change Composite")

In [5]:
# Step 4. Apply speckle filter
# Create a function to convert from dB to Natural
def toNatural(image):
  return ee.Image(10.0).pow(image.select(0).divide(10.0))

# Create a function to convert from Natural to dB
def todB(image):
  return ee.Image(image).log10().multiply(10.0)

# Apply a Refined Lee Speckle filter as coded in the SNAP 3.0 S1TBX:
# Adapted by Guido Lemoine

def RefinedLee(image):
  # Image must be in natural units, i.e., not in dB.
  # Set up a 3x3 kernels
  # weights3 = ee.List.repeat(ee.List.repeat({'value': 1, 'count': 3}), 3)
  weights3 = ee.List.repeat(ee.List.repeat(1, 3), 3)
  kernel3 = ee.Kernel.fixed(width=3, height=3, weights=weights3, x=1, y=1, normalize=False)
  mean3 = image.reduceNeighborhood(reducer=ee.Reducer.mean(), kernel=kernel3)
  variance3 = image.reduceNeighborhood(reducer=ee.Reducer.variance(), kernel=kernel3)
 
  # Use a sample of the 3x3 windows inside a 7x7 window sto determine gradients
  # and directions
  sample_weights = ee.List([[0, 0, 0, 0, 0, 0, 0],
                                [0, 1, 0, 1, 0, 1, 0],
                                [0, 0, 0, 0, 0, 0, 0],
                                [0, 1, 0, 1, 0, 1, 0],
                                [0, 0, 0, 0, 0, 0, 0],
                                [0, 1, 0, 1, 0, 1, 0],
                                [0, 0, 0, 0, 0, 0, 0]])
  sample_kernel = ee.Kernel.fixed(width=7, height=7, weights=sample_weights, x=3, y=3, normalize=False)                              

  # Calculate the mean and variance for the sampled windows and store as 9 bands
  sample_mean = mean3.neighborhoodToBands(sample_kernel)
  sample_var = variance3.neighborhoodToBands(sample_kernel)

  # Determine the 4 gradients for the sampled windows
  gradients = sample_mean.select(1).subtract(sample_mean.select(7).abs())
  gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
  gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
  gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

  # And find the maximum gradient amongst gradient bands
  max_gradient = gradients.reduce(ee.Reducer.max())

  # Create a mask for band pixels that are the maximum gradient
  gradmask = gradients.eq(max_gradient)

  # Duplicate gradmask bands: each gradient represents 2 directions
  gradmask = gradmask.addBands(gradmask)

  # Determine the 8 directions
  directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4) \
                              .subtract(sample_mean.select(7))).multiply(1)
  directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)) \
                         .gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
  directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)) \
                         .gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
  directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)) \
                         .gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))

  # The next 4 are the not() of the previous 4
  directions = directions.addBands(directions.select(0).Not().multiply(5))
  directions = directions.addBands(directions.select(1).Not().multiply(6))
  directions = directions.addBands(directions.select(2).Not().multiply(7))
  directions = directions.addBands(directions.select(3).Not().multiply(8))

  # Mask all values that are not 1-8
  directions = directions.updateMask(gradmask)

  # "collapse" the stack into a single band image (due to masking, each pixel has
  # just one value (1-8) in it's directional band, and is otherwise masked)
  directions = directions.reduce(ee.Reducer.sum())

  # pal = ["ffffff", "ff0000", "ffff00", "00ff00", "00ffff", "0000ff", "ff00ff", "000000"]
  # Map.addLayer(directions.reduce(ee.Reducer.sum()), {min: 1, max: 8, palette: pal}, "Directions", False)

  sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

  # Calculate localNoiseVariance
  sigmaV = sample_stats.toArray().arraySort().arraySlice(0, 0, 5) \
                           .arrayReduce(ee.Reducer.mean(), [0])

  # Set up the 7*7 kernels for directional statistics
  rect_weights = ee.List.repeat(ee.List.repeat(0, 7), 3) \
                       .cat(ee.List.repeat(ee.List.repeat(1, 7), 4))

  diag_weights = ee.List([[1, 0, 0, 0, 0, 0, 0],
                              [1, 1, 0, 0, 0, 0, 0],
                              [1, 1, 1, 0, 0, 0, 0],
                              [1, 1, 1, 1, 0, 0, 0],
                              [1, 1, 1, 1, 1, 0, 0],
                              [1, 1, 1, 1, 1, 1, 0],
                              [1, 1, 1, 1, 1, 1, 1]])

  rect_kernel = ee.Kernel.fixed(7, 7, rect_weights, 3, 3, False)
  diag_kernel = ee.Kernel.fixed(7, 7, diag_weights, 3, 3, False)

  # Create stacks for mean and variance using the original kernels. Mask with
  # relevant direction.
  dir_mean = image.reduceNeighborhood(ee.Reducer.mean(), rect_kernel) \
                      .updateMask(directions.eq(1))
  dir_var = image.reduceNeighborhood(ee.Reducer.variance(), rect_kernel) \
                     .updateMask(directions.eq(1))
  dir_mean = dir_mean.addBands(image.reduceNeighborhood(ee.Reducer.mean(), diag_kernel) \
                      .updateMask(directions.eq(2)))
  dir_= dir_var.addBands(image.reduceNeighborhood(ee.Reducer.variance(), diag_kernel) \
                      .updateMask(directions.eq(2)))

  # Add the bands for rotated kernels
  for i in range(1,4, 1):
    dir_mean = dir_mean.addBands(image.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)) \
                       .updateMask(directions.eq(2*i+1)))
    dir_= dir_var.addBands(image.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)) \
                       .updateMask(directions.eq(2*i+1)))
    dir_mean = dir_mean.addBands(image.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)) \
                       .updateMask(directions.eq(2*i+2)))
    dir_= dir_var.addBands(image.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)) \
                       .updateMask(directions.eq(2*i+2)))

  # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional
  #   band, and is otherwise masked)
  dir_mean = dir_mean.reduce(ee.Reducer.sum())
  dir_= dir_var.reduce(ee.Reducer.sum())

  # A finally generate the filtered value
  varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)) \
                    .divide(sigmaV.add(1.0))
  b = varX.divide(dir_var)

  result = dir_mean.add(b.multiply(image.subtract(dir_mean)))

  return (result.arrayFlatten([["sum"]]))

# Apply the Speckle filter on the before and after image
beforeFiltered = ee.Image(todB(RefinedLee(toNatural(beforeImage.select("VH")))))
afterFiltered = ee.Image(todB(RefinedLee(toNatural(afterImage.select("VH")))))

Map.addLayer(beforeFiltered, {'min':-25, 'max':0}, "VH before Speckle Filter", False)
Map.addLayer(afterFiltered, {'min':-25, 'max':0}, "VH after Speckle Filter", False)

In [6]:
# Step 5. Apply a threshold
difference = afterFiltered.divide(beforeFiltered)
# Define a threshold
diffThreshold = 1.25
# Initial estimate of flooded pixels
flooded = difference.gt(diffThreshold).rename("water").selfMask()
Map.addLayer(flooded, {'min':0, 'max':1, 'palette': ["orange"]}, "Initial Flood Area", False)

In [7]:
# Step 6. Apply masks
# Mask out area with permanent/semi-permanent water
permanentWater = gsw.select("seasonality").gte(5).clip(roi)
Map.addLayer(permanentWater.selfMask(), {'min':0, 'max':1, 'palette': ["blue"]}, "Permanent Water", False)

# GSW data is masked in non-water areas. Set it to 0 using unmask().
# Invert the image to set all non-permanent region to 1
permanentWaterMask = permanentWater.unmask(0).Not()
flooded = flooded.updateMask(permanentWaterMask)

# Mask out areas with more than 5 degree slope using the SRTM DEM
slopeThreshold = 5
slope = ee.Terrain.slope(srtm.clip(roi))
steepAreas = slope.gt(slopeThreshold)
slopeMask = steepAreas.Not()
Map.addLayer(slope, {'min':0, 'max':1, 'palette':["cyan"]}, "Steep Areas", False)

In [8]:
#Step 7: Remove Isolated Pixels
connectedPixelThreshold = 20
connections = flooded.connectedPixelCount(25)
disconnectedAreas = connections.lt(connectedPixelThreshold)
disconnectedAreaMask = disconnectedAreas.Not()
Map.addLayer(disconnectedAreas.selfMask(), {'min':0, 'max':1, 'palette': ["yellow"]}, "Disconnected Areas", False)
flooded = flooded.updateMask(disconnectedAreaMask)
Map.addLayer(flooded, {'min':0, 'max':1, 'palette': ["red"]}, "Flooded Areas")
flooded = flooded.updateMask(slopeMask)
Map

Map(center=[25.13268158688883, 88.08699823517007], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
# Calculate the area of flooded pixels
area_flooded = flooded.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=flooded.geometry(),
    scale=30,
    maxPixels=1e13
)

# Print the result
print("Area of flooded pixels:", area_flooded.getInfo())

Area of flooded pixels: {'water': 0}
